# TecoGAN

|Original| TecoGAN|
|:-------:|:-------------:|
|<a href="url"><img src='https://github.com/sony/nnabla-examples/raw/master/GANs/tecogan/results/cropped_lr_city.gif' width=250></a>|<a href="url"><img src='https://github.com/sony/nnabla-examples/raw/master/GANs/tecogan/results/cropped_sr_city.gif' width=250></a>|
|<a href="url"><img src='https://github.com/sony/nnabla-examples/raw/master/GANs/tecogan/results/cropped_lr_bridge.gif' width=250></a>|<a href="url"><img src='https://github.com/sony/nnabla-examples/raw/master/GANs/tecogan/results/cropped_sr_bridge.gif' width=250></a>|

This example interactively demonstrates TecoGAN from the paper [LEARNING TEMPORAL COHERENCE VIA SELF-SUPERVISION FOR GAN-BASED VIDEO GENERATION](https://arxiv.org/pdf/1811.09393.pdf), a model for video super-resolution that exploits temporal self-supervision. 

By leveraging temporal adversarial learning with a novel ping-pong loss, the model is able to generate results free of temporal artifacts while preserving spatial details.

# Preparation
Let's start by installing nnabla and accessing [nnabla-examples repository](https://github.com/sony/nnabla-examples). If you're running on Colab, make sure that your Runtime setting is set as GPU, which can be set up from the top menu (Runtime → change runtime type), and make sure to click **Connect** on the top right-hand side of the screen before you start.

In [ ]:
!pip install nnabla-ext-cuda100
!pip install ffmpeg
!git clone https://github.com/sony/nnabla-examples.git
%cd nnabla-examples/GANs/tecogan

Let's also download the pre-trained parameters, and a subset of test dataset provided by the author of the paper. This may take a few minutes.

In [ ]:
!wget https://nnabla.org/pretrained-models/nnabla-examples/GANs/tecogan/tecogan_model.h5
!wget https://ge.in.tum.de/download/data/TecoGAN/vid3_LR.zip
!unzip vid3_LR.zip

Let's also download the pre-trained weight parameters.

# Super-Resolution
Now, let's apply TecoGAN to one of the low-resolution scenes from the downloaded dataset. 

Running the following cell will run super-resolution with TecoGAN to the scene, and make two videos out of the original low-resolution frames and generated high-resolution frames.

In [ ]:
!python generate.py --model tecogan_model.h5 --input-dir-lr walk --output-dir results/walk
!ffmpeg -i results/walk/output_%04d.png -r 24/1 -y hr.mp4
!ffmpeg -i walk/%04d.png -r 24/1 -y lr.mp4

We're pretty much done, but we want to check the results with our own eyes, right?

Let's run the following cell first, which will help us play videos.



In [ ]:
from IPython.display import HTML
from base64 import b64encode


def play_video(filename, height=512, width=512):
    mp4 = open(filename, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML(f"""
    <video width={width} height={height} controls>
          <source src={data_url} type="video/mp4">
    </video>""")

Let's play the low resolution version first.

In [ ]:
play_video("lr.mp4")

Now let's play the high resolution version of it, rendered by TecoGAN!
You should be able to clearly see the difference.

In [ ]:
play_video("hr.mp4")

Did you notice the clear enhancement? :)

# Do it with your own video!

You can also do it with your own video! Run the following cell to upload your own video. You do not have to upload low resolution video, since we will convert it to low resolution, but try not to upload videos that are too long! We recommend videos of **less than 5 seconds** for stable demo on colab.

In [ ]:
from google.colab import files

video = files.upload()

Let's rename the uploaded video file for convenience.

In [ ]:
import os
ext = os.path.splitext(list(video.keys())[-1])[-1]
os.rename(list(video.keys())[-1], "input_video{}".format(ext)) 
input_video = "input_video" + ext

We will now extract frames from your uploaded video and subsequently make a low resolution version of it. 

We will convert it to resolution aspect ratio of width 200 and corresponding height, but if you want to specify your custom ratio, modify the part `scale=200:$new_height` in the cell below. Just make sure that both height and width are multiples of 4, and if aspect ratio is too high, it may result in memory error.

In [ ]:
width=!ffprobe -v error -select_streams v:0 -show_entries stream=width -of csv=p=0 input_video.mp4
height=!ffprobe -v error -select_streams v:0 -show_entries stream=height -of csv=p=0 input_video.mp4
new_height = int(height[0])*200 // int(width[0])
new_height = 4*round(new_height/4)
!mkdir -p frames/input_video
!ffmpeg -i $input_video -vf "fps=10, scale=200:$new_height" frames/input_video/frame_%04d.png
!ffmpeg -i frames/input_video/frame_%04d.png -r 24/1 -y user_video_lr.mp4

Now, let's apply super-resolution with TecoGAN and make a video out of it.

In [ ]:
!python generate.py --model tecogan_model.h5 --input-dir-lr frames/input_video/ --output-dir results/user
!ffmpeg -i results/user/output_frame_%04d.png -r 24/1 -y user_video_hr.mp4

Let's check the results!

First, let's check the low resolution version of your video.

In [ ]:
play_video("user_video_lr.mp4")

OK, it was a low resolution video that you proably don't feel like watching.

Now let's see the high resolution version of it rendered by TecoGAN!

In [ ]:
play_video("user_video_hr.mp4")

Hope you enjoyed it!